In [1]:
"""
This program tests Affine+Ditrortion35 model
on artificially generated data (pairs of star coordinates)
""";

In [2]:
from pylab import *
from PIL import Image, ImageDraw
from functools import partial

In [3]:
# for random consistency
np.random.seed(707666)

Create NUM_STAR_PAIRS (at least 5 because it's minimum needed for affine+distortion35 model)   
pairs of stars with coordinates in range from (MIN_X, MIN_Y) to (MAX_X, MAX_Y)
with OFFSET from edges

In [4]:
NUM_STAR_PAIRS = 20
N = NUM_STAR_PAIRS

MIN_X = 0
MAX_X = 2000
MIN_Y = 0
MAX_Y = 2000

OFFSET = 250

width = abs(MAX_X - MIN_X)
height = abs(MAX_Y - MIN_Y)

xCenter = (MAX_X - MIN_X) // 2
yCenter = (MAX_Y - MIN_Y) // 2


leftX = randint(low = MIN_X + OFFSET,
                high = MAX_X - OFFSET,
                size=NUM_STAR_PAIRS)
leftY = randint(low = MIN_Y + OFFSET,
                high = MAX_Y - OFFSET,
                size=NUM_STAR_PAIRS)



print('''\
Left X: {}
Left Y: {}
'''.format(leftX, leftY)
)

Left X: [1217 1731  516 1681  820 1276  513 1533  620  848 1725 1179 1335 1247 1043
 1114 1000  722 1027 1042]
Left Y: [ 570  718  598 1637  973 1092  740 1494 1085  812 1483 1274  420 1737  469
 1379  693 1170 1616 1655]



In [5]:
ELL_RAD = 3

# Draw star pairs
scatterOriginal = Image.new('RGB', (width, height), 'lightgray')

draw = ImageDraw.Draw(scatterOriginal)

# Central point
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')

for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((leftX[i] - ELL_RAD, leftY[i] - ELL_RAD, 
                  leftX[i] + ELL_RAD, leftY[i] + ELL_RAD), fill='blue')


In [6]:
scatterOriginal.save('orig.png')

Create affine transformed pairs of stars coordinates  
with affine coeffincients  
(a,b,  
 c,d) -- for rotation matrix  
(e,f) -- for transition (shift)   

In [7]:
def affine_transform(xy, coeffs=(1,0,0,1,0,0)):
    assert coeffs != (1,0,0,1,0,0)
        
    _a, _b, _c, _d, _e, _f = coeffs
    x, y = xy
    return [
        _a * x + _b * y + _e,
        _c * x + _d * y + _f
    ]

In [8]:
A_COEFF = 0.9951
B_COEFF = 0.0070
C_COEFF = -0.0118
D_COEFF = 1.0023
E_COEFF = 241.96
F_COEFF = 50.92
AFFINE_COEFFS = (A_COEFF, B_COEFF, C_COEFF, D_COEFF, E_COEFF, F_COEFF)

affinePerfect = partial(affine_transform, coeffs=AFFINE_COEFFS)

rightCoords = array(list(map(affinePerfect, zip(leftX, leftY))))
rightX = rightCoords[:, 0]
rightY = rightCoords[:, 1]

print('''\
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

Right X: [ 1456.9867  1969.5041   759.6176  1926.1821  1064.753   1519.3516
   757.6263  1777.9063   866.517   1091.4888  1968.8885  1424.1009
  1573.3585  1495.0087  1283.1323  1360.1544  1241.911    968.6122
  1275.2397  1290.4392]
Right Y: [  607.8704   750.1456   644.2066  1671.8493  1016.4819  1130.3748
   786.5686  1530.2668  1131.0995   854.7812  1516.9759  1313.938    456.133
  1777.2005   508.6913  1419.9465   733.7139  1215.0914  1658.5182
  1697.4309]



Create distorted pairs of stars coordinates  
with distortion coefficients  
eps1, eps2 -- for 3rd order distortion  
eps3, eps4 -- for 5th order distortion  

In [9]:
def distort_transform(xy, coeffs=(0,0)):
    assert coeffs != (0,0)
    
    # eps1, eps3 -- for left img
    # eps2, eps4 -- for right img
    _eps1_or_eps2, _eps3_or_eps4  = coeffs
    
    x, y = xy
    
    # squared distance from center to (x, y) point
    _r = (x - xCenter) ** 2 + (y - yCenter) ** 2
    
    return [
        x + (x - xCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 ),
        y + (y - yCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 )
    ]

In [10]:
EPS1 = 3e-8
EPS2 = 4e-8
EPS3 = 2e-14
EPS4 = -2e-14

DISTORT_COEFFS_LEFT = (EPS1, EPS3)
DISTORT_COEFFS_RIGHT = (EPS2, EPS4)

distortLeft = partial(distort_transform, coeffs=DISTORT_COEFFS_LEFT)
distortRight = partial(distort_transform, coeffs=DISTORT_COEFFS_RIGHT)

leftCoords = array(list(map(distortLeft, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]

rightCoords = array(list(map(distortRight, zip(rightX, rightY))))
rightX = rightCoords[:, 0]
rightY = rightCoords[:, 1]


print('''\
Left X: {}
Left Y: {}
'''.format(leftX, leftY)
)
print('''\
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

Left X: [ 1218.7438224   1749.97211513   508.73520705  1709.06234047   819.81715229
  1276.74036409   507.64263335  1544.41796348   618.09674993   847.72309197
  1749.85766116  1179.6162954   1340.85715267  1253.2802124   1043.43538628
  1114.59163868  1000.           721.05173621  1027.38600179  1042.69868064]
Left Y: [  566.54449939   710.68107187   591.96601908  1663.24920834   972.97257284
  1092.24678803   737.13980425  1504.58250273  1085.42572699   811.65750849
  1499.5603453   1274.94337954   409.85925807  1755.73893334   463.62348571
  1380.96693913   692.07742586  1170.57987354  1624.80655941  1665.89609088]

Right X: [ 1462.41318295  1988.89407351   758.00823016  1942.93485585  1064.76453805
  1524.4540932    756.66786772  1793.26336998   866.33336945  1091.59501774
  1987.44388305  1428.16650377  1583.20906556  1504.68336202  1286.18844744
  1363.88888388  1243.08236458   968.55427916  1279.41957097  1295.17775699]
Right Y: [  603.21406189   745.14853996   641.82454887  168

Add random noise (gaussian with std == NOISE_STD)

In [11]:
NOISE_STD = 1
NOISE_MEAN = 0

leftX += normal(loc=NOISE_MEAN,
                scale=NOISE_STD,
                size=leftX.shape)

leftY += normal(loc=NOISE_MEAN,
                scale=NOISE_STD,
                size=leftY.shape)


rightX += normal(loc=NOISE_MEAN,
                scale=NOISE_STD,
                size=rightX.shape)

rightY += normal(loc=NOISE_MEAN,
                scale=NOISE_STD,
                size=rightY.shape)

In [12]:
# inputLeftX, inputLeftY, inputRightX, inputRightY are coordinates we get from measuring system
inputLeftX = leftX
inputLeftY = leftY

inputRightX = rightX
inputRightY = rightY

In [13]:
def correct_distort(xy, coeffs=(0,0)):
    assert coeffs != (0,0)
    
    # eps1, eps3 -- for left img
    # eps2, eps4 -- for right img
    _eps1_or_eps2, _eps3_or_eps4  = coeffs
    
    x, y = xy
    
    # squared distance from center to (x, y) point
    _r = (x - xCenter) ** 2 + (y - yCenter) ** 2
    
    return [
        x - (x - xCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 ),
        y - (y - yCenter) * ( _r * _eps1_or_eps2 + (_r ** 2) * _eps3_or_eps4 )
    ]

Check out that if we do backward distort+affine transform then coordinates fit well.

In [14]:
# Backward distort

correctDistortLeftPerfect = partial(correct_distort, coeffs=DISTORT_COEFFS_LEFT)
leftCoords = array(list(map(correctDistortLeftPerfect, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]


correctDistortRightPerfect = partial(correct_distort, coeffs=DISTORT_COEFFS_RIGHT)
rightCoords = array(list(map(correctDistortRightPerfect, zip(rightX, rightY))))
rightX = rightCoords[:, 0]
rightY = rightCoords[:, 1]


# Backward affine
leftCoords = array(list(map(affinePerfect, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]

print('IF PERFECTLY FOUND COEFFICIENTS')
print('''Backward distort+affine Left:
Left X: {}
Left Y: {}
'''.format(leftX, leftY)
)

print('''Backward distort Right:
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

IF PERFECTLY FOUND COEFFICIENTS
Backward distort+affine Left:
Left X: [ 1457.22110812  1967.38648298   759.30304877  1922.24097919  1064.64927423
  1519.08772796   758.49308095  1775.6066839    865.85373783  1091.36045862
  1965.5407043   1424.61615835  1573.91478886  1492.28279469  1282.53652607
  1359.70225418  1241.87623741   969.27759202  1273.58586483  1289.41014898]
Left Y: [  608.12397792   752.06872422   643.92633483  1668.49804119  1016.54874411
  1131.57137137   788.91328701  1527.63412972  1131.40409206   856.54202577
  1514.48281178  1313.43299765   456.15068091  1776.6014066    509.94535375
  1418.56600445   734.68911552  1214.56933111  1658.94312653  1697.6208738 ]

Backward distort Right:
Right X: [ 1457.04009619  1969.0075761    759.65929639  1925.57689135  1064.37700427
  1519.61122512   755.9952595   1776.59429569   866.11515907  1091.66808014
  1968.12261864  1423.87139981  1573.455849    1493.45405089  1283.0650448
  1359.95778767  1241.68525074   968.57978727  1275

In [15]:
delX = abs(leftX - rightX)
delY = abs(leftY - rightY)
print("delX:", delX)
print("delY:", delY)

sigSqr = 1.0 / N * sum(delX**2 + delY**2)
mX = max(delX)
mY = max(delY)
m = max(mX, mY)

print("IF PERFECTLY FOUND COEFFICIENTS")
print("mX: %.4f mY: %.4f m: %.4f" % (mX, mY, m))
print("sigSqr: %.4f" % sigSqr)

delX: [ 0.18101194  1.62109312  0.35624762  3.33591216  0.27226996  0.52349717
  2.49782145  0.98761179  0.26142124  0.30762152  2.58191434  0.74475854
  0.45893986  1.1712562   0.52851874  0.2555335   0.19098667  0.69780475
  2.15503989  0.7787122 ]
delY: [ 0.02694586  1.75390105  0.12038763  2.8908792   0.91512563  1.24461156
  3.432414    3.37550095  1.24800576  2.35734269  2.84014901  1.93606847
  1.43244899  1.20157996  1.44312881  2.68116455  0.15591205  1.82594436
  1.46066645  1.12366911]
IF PERFECTLY FOUND COEFFICIENTS
mX: 3.3359 mY: 3.4324 m: 3.4324
sigSqr: 5.6013


### Test affine model

Calculate model coefficients

In [16]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [17]:
xi = np.zeros(2 * NUM_STAR_PAIRS)

for i in range(NUM_STAR_PAIRS): # fill the xi vector
    xi[2 * i] = rightX[i]
    xi[2 * i + 1] = rightY[i]

print('xi:\n', xi)

xi:
 [ 1462.63626688   603.35297643  1988.74674445   745.22861139   758.01716864
   641.37225151  1942.05132419  1683.33903689  1064.38828693  1015.63635841
  1524.868709    1134.15982975   755.00793265   784.61285505  1792.34308442
  1541.77813429   865.93094494  1130.33516997  1091.77561438   854.01362966
  1986.60830063  1527.20091173  1428.05162095  1318.47923688  1583.66162718
   447.92975158  1503.37425461  1790.98815596  1286.21111534   503.03956998
  1363.80598361  1425.75059051  1242.86589217   733.54973569   968.52080518
  1216.80149316  1280.0755024   1667.81796668  1295.09240885  1708.26646155]


In [18]:
k = 6 # num of coeff-s

z = np.zeros(k)
arr = np.zeros((2 * NUM_STAR_PAIRS, k)) # matrix A

for i in range(NUM_STAR_PAIRS): # fill the A matrix
    
    arr[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0]

    arr[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1]

    
p_arr = pinv(arr, rcond=1e-20)
z = np.dot(p_arr, xi)

print("""
Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f""" % tuple(z))
print("""
Perfect Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f""" % AFFINE_COEFFS)
print('cond(A): ', np.linalg.cond(arr))


Affine coefficients:
0.9903 0.0059 -0.0145 0.9993 
250.26 56.53

Perfect Affine coefficients:
0.9951 0.0070 -0.0118 1.0023 
241.96 50.92
cond(A):  5755.55856641


Calculate error metrics

In [19]:
"""
Align images and blend

a) Affine
""";

In [20]:
affine = partial(affine_transform, coeffs=tuple(z))

# Calc estimated (affine transformed) points
leftCoords = array(list(map(affine, zip(leftX, leftY))))

# Estimated coordinates
estLeftX = leftCoords[:, 0]
estLeftY = leftCoords[:, 1]


print('''Backward affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX, estLeftY)
)

print('''Right:
Right X: {}
Right Y: {}
'''.format(rightX, rightY)
)

Backward affine Left:
Left X: [ 1460.76052512  1987.02167788   756.80199694  1953.30720271  1067.72852806
  1520.75384618   758.01542565  1786.97441489   868.03649443  1094.38757508
  1991.84139688  1426.43129662  1581.43082949  1499.52682622  1285.68257859
  1361.69771907  1244.56985572   971.8537436   1275.57644609  1291.63520819]
Left Y: [  605.14028546   742.54862214   640.07033615  1694.85303548  1016.98092456
  1130.66936839   788.05714782  1535.68197119  1132.53359674   857.06784659
  1529.26623901  1312.95160741   445.9598136   1794.06557228   505.78310348
  1418.97680247   734.57254923  1215.29574358  1666.18373895  1706.99442255]

Right:
Right X: [ 1462.63626688  1988.74674445   758.01716864  1942.05132419  1064.38828693
  1524.868709     755.00793265  1792.34308442   865.93094494  1091.77561438
  1986.60830063  1428.05162095  1583.66162718  1503.37425461  1286.21111534
  1363.80598361  1242.86589217   968.52080518  1280.0755024   1295.09240885]
Right Y: [  603.35297643   745

1) $\Delta x_i, \Delta y_i, \; i = 1,N$

2) $\sigma^2 = \frac{1}{N} \sum\limits_{i=1}^{N} 
                \left( \Delta x_i^2 + \Delta y_i^2 \right)$

In [21]:
delX = abs(estLeftX - rightX)
delY = abs(estLeftY - rightY)
print("delX:", delX)
print("delY:", delY)

sigSqr = 1.0 / N * sum(delX**2 + delY**2)
mX = max(delX)
mY = max(delY)
m = max(mX, mY)

print("mX: %.4f mY: %.4f m: %.4f" % (mX, mY, m))
print("sigSqr: %.4f" % sigSqr)

delX: [  1.87574176   1.72506657   1.2151717   11.25587851   3.34024113
   4.11486282   3.00749301   5.36866953   2.10554948   2.6119607
   5.23309625   1.62032433   2.23079769   3.8474284    0.52853675
   2.10826454   1.70396356   3.33293842   4.49905631   3.45720066]
delY: [  1.78730903   2.67998925   1.30191536  11.51399859   1.34456615
   3.49046136   3.44429276   6.0961631    2.19842677   3.05421693
   2.06532728   5.52762947   1.96993798   3.07741632   2.7435335
   6.77378804   1.02281355   1.50574958   1.63422773   1.272039  ]
mX: 11.2559 mY: 11.5140 m: 11.5140
sigSqr: 32.1803


Plot aligned star pairs

In [22]:
scatter = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((estLeftX[i] - ELL_RAD, estLeftY[i] - ELL_RAD, 
                  estLeftX[i] + ELL_RAD, estLeftY[i] + ELL_RAD), fill='red')
    
    draw.ellipse((rightX[i] - ELL_RAD, rightY[i] - ELL_RAD, 
                  rightX[i] + ELL_RAD, rightY[i] + ELL_RAD), fill='blue')

In [23]:
scatter.save('000.png')

### Test affine+distortion3 model

Calculate model coefficients

In [24]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

Calculate error metrics

Plot aligned star pairs

### Test affine+distortion35 model

Calculate model coefficients

In [25]:
leftX = inputLeftX
leftY = inputLeftY
rightX = inputRightX
rightY = inputRightY

In [26]:
"""
c) Affine + Ditortion 3rd, 5th orders 
  (at least 5 stars)
""";

In [27]:
k35 = 10

z35 = np.zeros(k35)
arr35 = np.zeros((2 * N, k35)) # matrix A

for i in range(N): # fill the A matrix
    dist_l = (leftX[i] - xCenter) ** 2 + (leftY[i] - yCenter) ** 2
    dist_r = (rightX[i] - xCenter) ** 2 + (rightY[i] - yCenter) ** 2

    zx1 = (leftX[i] - xCenter) * dist_l
    zx2 = (rightX[i] - xCenter) * dist_r
    wx1 = (leftX[i] - xCenter) * dist_l ** 2
    wx2 = (rightX[i] - xCenter) * dist_r ** 2

    arr35[2 * i] = [leftX[i], leftY[i], 0, 0, 1, 0, -zx1, zx2, -wx1, wx2]

    zy1 = (leftY[i] - yCenter) * dist_l
    zy2 = (rightY[i] - yCenter) * dist_r
    wy1 = (leftY[i] - yCenter) * dist_l ** 2
    wy2 = (rightY[i] - yCenter) * dist_r ** 2

    arr35[2 * i + 1] = [0, 0, leftX[i], leftY[i], 0, 1, -zy1, zy2, -wy1, wy2]


In [28]:
p_arr35 = pinv(arr35, rcond=1e-20)
z35 = np.dot(p_arr35, xi)


print("""
Affine coefficients + Ditortion 3rd, 5th orders:

%.4f %.4f %.4f %.4f 
%.2f %.2f 
%.2e %.2e 
%.2e %.2e""" % tuple(z35))

print("""
Perfect Affine coefficients:
%.4f %.4f %.4f %.4f 
%.2f %.2f
%.2e %.2e
%.2e %.2e""" % tuple( list(AFFINE_COEFFS) + [EPS1, EPS2, EPS3, EPS4] ) )


print('cond(A): ', np.linalg.cond(arr35))


Affine coefficients + Ditortion 3rd, 5th orders:

1.0005 0.0077 -0.0096 1.0096 
236.76 41.54 
3.11e-08 2.30e-08 
4.02e-14 -4.97e-16

Perfect Affine coefficients:
0.9951 0.0070 -0.0118 1.0023 
241.96 50.92
3.00e-08 4.00e-08
2.00e-14 -2.00e-14
cond(A):  7.83251351581e+15


Calculate error metrics

In [29]:
"""
c) Affine + Ditortion3,5
""";

In [30]:
a = float(z35[0])
b = float(z35[1])
c = float(z35[2])
d = float(z35[3])
e = float(z35[4])
f = float(z35[5])

eps1 = float(z35[6])
eps2 = float(z35[7])
eps3 = float(z35[8])
eps4 = float(z35[9])

In [31]:
# Backward distort

correctDistortLeft = partial(correct_distort, coeffs=(eps1, eps3))
leftCoords = array(list(map(correctDistortLeft, zip(leftX, leftY))))
leftX = leftCoords[:, 0]
leftY = leftCoords[:, 1]


correctDistortRight = partial(correct_distort, coeffs=(eps2, eps4))
rightCoords = array(list(map(correctDistortRight, zip(rightX, rightY))))
estRightX35 = rightCoords[:, 0]
estRightY35 = rightCoords[:, 1]


# Backward affine
affine = partial(affine_transform, coeffs=(a,b,c,d,e,f))


leftCoords = array(list(map(affine, zip(leftX, leftY))))
estLeftX35 = leftCoords[:, 0]
estLeftY35 = leftCoords[:, 1]


print('''Backward distort+affine Left:
Left X: {}
Left Y: {}
'''.format(estLeftX35, estLeftY35)
)

print('''Backward distort Right:
Right X: {}
Right Y: {}
'''.format(estRightX35, estRightY35)
)

Backward distort+affine Left:
Left X: [ 1458.67737653  1965.19769802   759.17794296  1913.53051684  1064.5296576
  1521.43330091   757.68061009  1776.02176432   864.95934898  1091.29740886
  1959.92535163  1426.56189365  1574.5594026   1492.66782988  1283.20272305
  1361.34532009  1242.53733946   968.84581066  1274.87891746  1290.70977485]
Left Y: [  606.13156331   754.37564761   641.57678758  1661.97636499  1016.03140009
  1132.88788736   786.64108026  1529.08579062  1131.21291504   854.94701057
  1512.56067243  1315.78131241   455.64554536  1775.80865495   507.31941641
  1421.37198852   732.61163771  1215.21243366  1661.32450311  1699.51586812]

Backward distort Right:
Right X: [ 1458.71525095  1965.56557745   759.05498915  1913.55548731  1064.38178371
  1521.34713291   755.60635643  1775.88216872   866.03870482  1091.71286978
  1958.97258873  1425.26549505  1575.11531253  1493.38729156  1284.06082642
  1361.1987497   1242.1416513    968.55552853  1276.73456692  1291.14631397]
Right 

In [32]:
delX35 = abs(estLeftX35 - estRightX35)
delY35 = abs(estLeftY35 - estRightY35)
print("delX35:", delX35)
print("delY35:", delY35)

sigSqr35 = 1.0 / N * sum(delX35**2 + delY35**2)
mX35 = max(delX35)
mY35 = max(delY35)
m35 = max(mX35, mY35)

print("mX35: %.4f mY35: %.4f m35: %.4f" % (mX35, mY35, m35))
print("sigSqr35: %.4f" % sigSqr35)

delX35: [ 0.03787442  0.36787943  0.1229538   0.02497047  0.14787388  0.08616799
  2.07425366  0.1395956   1.07935584  0.41546092  0.9527629   1.29639859
  0.55590993  0.71946168  0.85810337  0.14657039  0.39568815  0.29028212
  1.85564946  0.43653912]
delY35: [ 0.58314546  3.17392112  1.33355335  0.69254785  0.39662095  0.37180481
  1.50211516  1.43691106  0.98250378  0.83357383  0.12709288  0.62498933
  0.36794144  0.51373204  0.54620909  1.32743892  1.73266888  1.34991504
  1.47273383  0.72063177]
mX35: 2.0743 mY35: 3.1739 m35: 3.1739
sigSqr35: 2.1608


Plot aligned star pairs

In [33]:
scatter35 = Image.new('RGB', (width, height), 'lightgray')


draw = ImageDraw.Draw(scatter35)
draw.ellipse((xCenter - ELL_RAD, yCenter - ELL_RAD, 
              xCenter + ELL_RAD, yCenter + ELL_RAD), fill='darkgreen')


for i in range(NUM_STAR_PAIRS): # draw star points
    draw.ellipse((estLeftX35[i] - ELL_RAD, estLeftY35[i] - ELL_RAD, 
                  estLeftX35[i] + ELL_RAD, estLeftY35[i] + ELL_RAD), fill='red')
    
    draw.ellipse((estRightX35[i] - ELL_RAD, estRightY35[i] - ELL_RAD, 
                  estRightX35[i] + ELL_RAD, estRightY35[i] + ELL_RAD), fill='blue')


In [34]:
scatter35.save('035.png')